# IFC Rasterization for Efficient Spatial Analysis

## Intro

### Import libraries and the IFC model:

In [25]:
import pyvista as pv
import numpy as np
from typing import Any, Tuple
import ifcopenshell
import logging
from importlib import reload 
import multiprocessing
import ifcopenshell
import ifcopenshell.geom
import time
import vtk
ifc_file = ifcopenshell.open(r"IFC Files\Project1.ifc")
#ifc_file = ifcopenshell.open(r"IFC Files\Duplex.ifc")

### Logging setup:

## Functions

In [26]:
def to_vtk_faces(faces : Tuple[tuple]) -> np.ndarray:
    faces=np.array(faces, dtype=np.int16)
    num_insertions = (len(faces) - 1) // 3
    # Generate an array of indices for insertions
    indices = np.arange(3, 3 * (num_insertions + 1), 3)
    indices = np.insert(indices, 0, 0)
    faces = np.insert(faces, indices, 3)
    return faces

def vtk_block_by_building_element(ifc_file):
    building_elements = ifc_file.by_type("IfcBuildingElement")
    settings = ifcopenshell.geom.settings()
    settings.set(settings.USE_WORLD_COORDS, True)
    settings.set(settings.APPLY_DEFAULT_MATERIALS, True)
    iterator = ifcopenshell.geom.iterator(settings, ifc_file, multiprocessing.cpu_count())
    multiblock = pv.MultiBlock()
    element_information = {} # Dictionary to hold element information
    exclude_list = ["IfcSpace", "IfcOpeningElement"]

    if iterator.initialize():
        while True:
            shape = iterator.get()
            if shape.type not in exclude_list:
                element = ifc_file.by_guid(shape.guid)
                           
                faces = shape.geometry.faces
                verts = shape.geometry.verts
                poly_data = pv.PolyData(list(verts), to_vtk_faces(faces))
                multiblock.append(poly_data)
                  
                if element in building_elements:
                #print(element.all_attributes()) --> why doesn't it work?
                        
                    element_information[shape.guid] = {
                    "Geo": poly_data, 
                    "Type": shape.type,
                    "Name": element.Name,
                    "Description": element.Description
                    }
                            
                       
            if not iterator.next():
                break
                
    return multiblock, element_information

def create_uniform_grid(bounds, voxel_size):
    """Create a uniform grid within the given bounds."""
    x = np.arange(bounds[0], bounds[1] + voxel_size, voxel_size)
    y = np.arange(bounds[2], bounds[3] + voxel_size, voxel_size)
    z = np.arange(bounds[4], bounds[5] + voxel_size, voxel_size)
    return pv.StructuredGrid(*np.meshgrid(x, y, z))

def calculate_offset_points(cell_center, voxel_size):
    offset = voxel_size / 4
    offset_points = [
        cell_center + np.array([offset, 0, 0]),  # Point offset in the +x direction
        cell_center - np.array([offset, 0, 0]),  # Point offset in the -x direction
        cell_center + np.array([0, offset, 0]),  # Point offset in the +y direction
        cell_center - np.array([0, offset, 0]),  # Point offset in the -y direction
        cell_center + np.array([0, 0, offset]),  # Point offset in the +z direction
        cell_center - np.array([0, 0, offset])   # Point offset in the -z direction
    ]
    return offset_points


In [27]:
def voxelize_space(meshes, voxel_size, mesh_info):
    """Voxelize space and check intersections with given mesh."""
    grid = create_uniform_grid(meshes.bounds, voxel_size)

    num_points = grid.cell_centers().n_points
    
    mask = np.zeros(num_points, dtype=bool)

    cell_to_guid = {}

    print(f'Total number of voxels: {num_points}')

    select_enclosed = vtk.vtkSelectEnclosedPoints()
    select_enclosed_corners = vtk.vtkSelectEnclosedPoints()
    select_enclosed.SetInputData(grid.cell_centers())
    
    for guid, mesh_data in mesh_info.items():
        mesh = mesh_data['Geo']
        select_enclosed.SetSurfaceData(mesh)
        select_enclosed_corners.SetSurfaceData(mesh)
        select_enclosed.Update()
        
    for i in range(num_points):
        print(i)

        if not mask[i] and select_enclosed.IsInside(i):
            mask[i] = True
            cell_to_guid[i] = guid

        else:
            cell_center = grid.cell_centers().points[i]
            offset_points = calculate_offset_points(cell_center, voxel_size)
            print(offset_points)

            for ind, offset_point in enumerate(offset_points):
                offset_point_polydata = pv.PolyData(np.array([offset_point]))
                select_enclosed_corners.SetInputData(offset_point_polydata)
                select_enclosed_corners.Update()
                print(select_enclosed_corners)
                if not mask[i] and select_enclosed_corners.IsInside(ind):
                    mask[i] = True
                    break
                            
    return grid, mask, cell_to_guid

## Voxelization

In [28]:
all_meshes, info = vtk_block_by_building_element(ifc_file)

p = pv.Plotter()

start_time = time.time()

voxel_size = 0.5

# Voxelize the entire space of the combined mesh
grid, mask2, cell_info = voxelize_space(all_meshes, voxel_size, info)

# Prepare to store cell-IFC-object mappings
'''cell_data = {}

for cell_id, guid in cell_info.items():
    cell_data[cell_id] = info[guid]
'''
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken to voxelize: {elapsed_time:.4f} seconds")

# Visualization
p.add_mesh(grid, opacity=0.3, show_edges=True)
 
p.add_mesh(grid.extract_cells(np.where(mask2)[0]), color="red", opacity=0.5)

p.show()


Total number of voxels: 800
0
[pyvista_ndarray([-1.93190329, -2.4290849 , -0.23      ]), pyvista_ndarray([-2.18190329, -2.4290849 , -0.23      ]), pyvista_ndarray([-2.05690329, -2.3040849 , -0.23      ]), pyvista_ndarray([-2.05690329, -2.5540849 , -0.23      ]), pyvista_ndarray([-2.05690329, -2.4290849 , -0.105     ]), pyvista_ndarray([-2.05690329, -2.4290849 , -0.355     ])]
vtkSelectEnclosedPoints (000001E1EF252950)
  Debug: Off
  Modified Time: 96133203
  Reference Count: 2
  Registered Events: (none)
  Executive: 000001E1F0F63540
  ErrorCode: No error
  Information: 000001E1F12A2330
  AbortExecute: Off
  Progress: 1
  Progress Text: (None)
  Check Surface: Off
  Inside Out: Off
  Tolerance: 0.0001


vtkSelectEnclosedPoints (000001E1EF252950)
  Debug: Off
  Modified Time: 96133513
  Reference Count: 2
  Registered Events: (none)
  Executive: 000001E1F0F63540
  ErrorCode: No error
  Information: 000001E1F12A2330
  AbortExecute: Off
  Progress: 1
  Progress Text: (None)
  Check Surfac

Widget(value="<iframe src='http://localhost:53577/index.html?ui=P_0x1e1f2040910_6&reconnect=auto' style='width…